# Accessing info with `cirq-superstaq`
This tutorial will cover the information you can access on your account and related jobs and backends using `cirq-superstaq`.

## Imports and API Token

As usual, we'll begin with importing requirements and setting up access to Superstaq. This tutorial uses `cirq-superstaq`, our Superstaq client for Cirq. You can install it and relevant dependencies by running `pip install cirq-superstaq`.

In [1]:
# Requirements to use cirq-superstaq
try:
    import cirq
    import cirq_superstaq as css
except ImportError:
    print("Installing cirq-superstaq...")
    %pip install --quiet 'cirq-superstaq[examples]'
    print("Installed cirq-superstaq.")
    print("You may need to restart the kernel to import newly installed packages.")
    import cirq
    import cirq_superstaq as css

Now, we instantiate a service provider in `cirq-superstaq` with `Service()`. Supply the Superstaq API token by providing the token as an argument of `css.Service()` or setting it as an environment variable (see [this guide](https://superstaq.readthedocs.io/en/latest/get_started/basics/basics_css.html#Set-up-access-to-Superstaq%E2%80%99s-API)).

In [2]:
service = css.Service()

## Account Information

The `service` class gives you a means to retrieve information regarding your Superstaq account. Currently, you can use `service` to retrieve your Superstaq balance.

In [3]:
service.get_balance()

'135.75 credits'

If are interested in increasing your balance or have more information on your user role, please reach out to us at superstaq@infleqtion.com or join our [Slack workspace](https://join.slack.com/t/superstaq/shared_invite/zt-1wr6eok5j-fMwB7dPEWGG~5S474xGhxw).

## Backend Information
In addition to account information, the`Service` object also gives you a list of all the devices and simulators to which you have access, as well as additional information about those backends.

* `get_targets()`: Retrieves a list of available backends
* `target_info("<target_name>")`: Retrieve information on your selected backend, such as number of qubits, native gate set, where `<target_name>` is the name of the desired backend

In [4]:
# service.get_targets()

{'compile-only': ['aqt_keysight_qpu', 'aqt_zurich_qpu', 'sandia_qscout_qpu'],
 'compile-and-run': ['aws_dm1_simulator',
  'aws_sv1_simulator',
  'aws_tn1_simulator',
  'cq_hilbert_qpu',
  'cq_hilbert_simulator',
  'ibmq_brisbane_qpu',
  'ibmq_extended-stabilizer_simulator',
  'ibmq_fake-athens_qpu',
  'ibmq_fake-lima_qpu',
  'ibmq_guadalupe_qpu',
  'ibmq_lagos_qpu',
  'ibmq_mps_simulator',
  'ibmq_nairobi_qpu',
  'ibmq_perth_qpu',
  'ibmq_qasm_simulator',
  'ibmq_stabilizer_simulator',
  'ibmq_statevector_simulator',
  'ionq_aria-1_qpu',
  'ionq_harmony_qpu',
  'ionq_ion_simulator',
  'qtm_h1-1_qpu',
  'qtm_h1-1e_simulator',
  'qtm_h2-1_qpu',
  'ss_unconstrained_simulator'],
 'unavailable': ['ionq_aria-2_qpu',
  'oxford_lucy_qpu',
  'rigetti_aspen-m-3_qpu'],
 'retired': ['rigetti_aspen-10_qpu',
  'rigetti_aspen-11_qpu',
  'rigetti_aspen-8_qpu',
  'rigetti_aspen-9_qpu',
  'rigetti_aspen-m-1_qpu',
  'rigetti_aspen-m-2_qpu']}

In [5]:
service.target_info("ibmq_lagos_qpu")  # retrieving information about the IBM Lagos device

{'num_qubits': 7,
 'target': 'ibmq_lagos_qpu',
 'coupling_map': [[0, 1],
  [1, 0],
  [1, 2],
  [1, 3],
  [2, 1],
  [3, 1],
  [3, 5],
  [4, 5],
  [5, 3],
  [5, 4],
  [5, 6],
  [6, 5]],
 'supports_midcircuit_measurement': True,
 'native_gate_set': ['cx', 'id', 'rz', 'sx', 'x'],
 'max_experiments': 900,
 'max_shots': 100000,
 'processor_type': {'family': 'Falcon', 'revision': '5.11', 'segment': 'H'},
 'open_pulse': True,
 'conditional': False}

## Job Information
Jobs submitted through Superstaq contain the following information:

* `job_id()`: Unique identifier for the job.
* `status()`: Status of the job (either Queued, Running, Done).
* `target()`: Device the job was run on.
* `counts()`: Counts from the result of the job run. Note this can return a list, so you must specify an `index` if you want to retrieve a specific counts dictionary (corresponding to the circuit with the same `index`).
* `num_qubits()`: Number of qubits for the job. Note this can return a list, so you must specify an `index` if you want to retrieve the corresponding number of qubits to a particular circuit. 
* `repetitions()`: Number of repetitions for the job.
* `input_circuits()`: Retrieves original (i.e., not compiled) circuit(s) for job. Note this returns a list and you must specify an `index` if you want to retrieve a single/specific circuit.
* `compiled_circuits()`: Retrieves compiled circuit(s) from submitted job. Note this returns a list and you must specify an `index` if you want to retrieve a single/specific circuit.

Note that jobs live in our database for a limited amount of time. Typically, they have a lifespan of 1 year.

In [6]:
# Creating a circuit using Cirq
qubits = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(qubits[0]),
    cirq.CNOT(qubits[0], qubits[1]),
    cirq.measure(qubits[0]),
    cirq.measure(qubits[1]),
)

# Submitting the circuit to the IBMQ QASM Simulator
job = service.create_job(
    circuit, repetitions=100, method="dry-run", target="ibmq_qasm_simulator"
)  # Specify "dry-run" as the method to submit & run a Superstaq simulation

In [7]:
job.job_id()

'49cbea8a-9aa8-4b09-ada1-8d7476287794'

In [8]:
job.status()

'Done'

In [9]:
job.target()

'ibmq_qasm_simulator'

In [10]:
job.counts(index=0)

{'00': 51, '11': 49}

In [11]:
job.num_qubits(index=0)

2

In [12]:
job.repetitions()

100

In [13]:
job.input_circuits(index=0)

0: ───H───@───M───
          │
1: ───────X───M───

In [14]:
job.compiled_circuits(index=0)

0: ───cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.0, lmda=1.0)───@───M───
                                                                          │
1: ───────────────────────────────────────────────────────────────────────X───M───

You may also retrieve the information described above on a previously submitted `cirq-superstaq` job with `service.get_job('<job_id>')`:

In [15]:
job_id = job.job_id()  # Here we use the job ID from above, but this can be any old job ID
job_old = service.get_job(job_id)
job_old.input_circuits(index=0)

0: ───H───@───M───
          │
1: ───────X───M───